# Description:

In [150]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.optim as optim


In [151]:
device='cuda:0' if torch.cuda.is_available() else 'cpu'


In [152]:
dataset=pd.read_csv('DailyDelhiClimateTrain.csv')
dataset['date']=pd.to_datetime(dataset['date'])

dataset=dataset[['date','meantemp','humidity','wind_speed','meanpressure']]


In [153]:
train_size=int(0.7*len(dataset))
train_data,val_data=dataset[:train_size],dataset[train_size:]
val_dates=dataset['date'][train_size:]


In [154]:
train_data=train_data.drop(columns=['date']).values
val_data=val_data.drop(columns=['date']).values


In [155]:
'''from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder1',OneHotEncoder(),[4])],remainder='passthrough')
train_data=ct.fit_transform(train_data)
test_data=ct.fit_transform(test_data)'''


"from sklearn.compose import ColumnTransformer\nfrom sklearn.preprocessing import OneHotEncoder\nct=ColumnTransformer(transformers=[('encoder1',OneHotEncoder(),[4])],remainder='passthrough')\ntrain_data=ct.fit_transform(train_data)\ntest_data=ct.fit_transform(test_data)"

In [156]:
print(train_data[0])
print(val_data[1])

[  10.           84.5           0.         1015.66666667]
[  24.75   56.      4.4  1012.25]


In [157]:
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler()
scaled_train_data=sc.fit_transform(train_data[:,0:])
scaled_test_data=sc.transform(val_data[:,0:])

In [158]:
print(scaled_train_data.shape)

(1023, 4)


In [159]:
def create_input(data,timestep=1):
    X,Y=[],[]
    for i in range(len(data)-timestep-1):
        X.append(scaled_train_data[i:(i+timestep)])
        Y.append(scaled_train_data[(i+timestep)])    
    return np.array(X),np.array(Y)


In [160]:
timestep=7
x_train,y_train=create_input(scaled_train_data,timestep)
x_test,y_test=create_input(scaled_test_data,timestep)

In [161]:
#x_train=x_train.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2])
#x_test=x_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2])

In [162]:
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_train=y_train.astype(np.float32)
y_test=y_test.astype(np.float32)

In [163]:
print(x_train.shape)

(1015, 7, 4)


In [164]:
x_train=torch.tensor(x_train).float()
x_test=torch.tensor(x_test).float()
y_train=torch.tensor(y_train).float()
y_test=torch.tensor(y_test).float()


In [165]:
print(x_train.shape)

torch.Size([1015, 7, 4])


In [166]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM,self).__init__()
        self.lstm=nn.LSTM(input_size=4,num_layers=2,hidden_size=128,batch_first=True,bidirectional=True)
        self.dropout=nn.Dropout(0.2)
        self.linear1=nn.Linear(128*2,64)
        self.linear2=nn.Linear(64,8)
        self.output_layer=nn.Linear(8,4)
    def forward(self,x):
        h0=torch.zeros(self.lstm.num_layers*2,x.size(0),self.lstm.hidden_size).to(device)
        c0=torch.zeros(self.lstm.num_layers*2,x.size(0),self.lstm.hidden_size).to(device)
        out,_=self.lstm(x,(h0,c0))
        out=self.dropout(out)
        out=self.linear1(out[:,-1,:])
        out=self.linear2(out)
        out=self.output_layer(out)
        return out



In [167]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
model=LSTM()
loss_fn=nn.MSELoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)
scheduler=ReduceLROnPlateau(optimizer,mode='min',factor=0.5,patience=5,verbose=True)

c:\Users\tejat\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [171]:
epochs=20
threshold=1
for epoch in range(1,epochs+1):
    model.train()
    y_logits=model(x_train).squeeze()
    y_pred=torch.round(torch.sigmoid(y_logits))
    #print(y_logits.shape)
    loss=loss_fn(y_logits,y_train)
    acc=torch.mean((torch.abs(y_train - y_pred) <= threshold).float(), dim=0)
    overall_accuracy = acc.mean().item() * 100  
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    test_pred=model(x_test)
    test_loss=loss_fn(test_pred,y_test)
    acc_test=torch.mean((torch.abs(y_test - test_pred) <= threshold).float(), dim=0)
    overall_test_accuracy = acc_test.mean().item() * 100
    if epoch % 1 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f} | Accuracy: {acc} | Overall Accuracy : {overall_accuracy} | Test Loss: {test_loss:.5f} | Test Accuracy :{acc_test} | Overall Test Accuracy : {overall_test_accuracy}")

Epoch: 1 | Loss: 0.00742 | Accuracy: tensor([1., 1., 1., 1.]) | Overall Accuracy : 100.0 | Test Loss: 0.00795 | Test Accuracy :tensor([1., 1., 1., 1.]) | Overall Test Accuracy : 100.0
Epoch: 2 | Loss: 0.00753 | Accuracy: tensor([1., 1., 1., 1.]) | Overall Accuracy : 100.0 | Test Loss: 0.00786 | Test Accuracy :tensor([1., 1., 1., 1.]) | Overall Test Accuracy : 100.0
Epoch: 3 | Loss: 0.00734 | Accuracy: tensor([1., 1., 1., 1.]) | Overall Accuracy : 100.0 | Test Loss: 0.00777 | Test Accuracy :tensor([1., 1., 1., 1.]) | Overall Test Accuracy : 100.0
Epoch: 4 | Loss: 0.00740 | Accuracy: tensor([1., 1., 1., 1.]) | Overall Accuracy : 100.0 | Test Loss: 0.00790 | Test Accuracy :tensor([1., 1., 1., 1.]) | Overall Test Accuracy : 100.0
Epoch: 5 | Loss: 0.00724 | Accuracy: tensor([1., 1., 1., 1.]) | Overall Accuracy : 100.0 | Test Loss: 0.00791 | Test Accuracy :tensor([1., 1., 1., 1.]) | Overall Test Accuracy : 100.0
Epoch: 6 | Loss: 0.00734 | Accuracy: tensor([1., 1., 1., 1.]) | Overall Accuracy